In [65]:
### Define EKF model
### All function that initalize the EKF

### EKF function as well as all other necessary functions

In [66]:
import numpy as np
from scipy.integrate import solve_ivp

In [67]:
def EKF(Xi, Pi, mu, DT, DO, t_data, t_steps, obsN):
    """
    Parameters
    ----------
    Xi = Inital Condition [6 by 1]
    Pi = Inital Covariance Matrix [6 by 6]
    mu = mass parameter
    DT = Target State Data [6 by size of t_data]
    DO = Observer State Data [6 by size of t_data]
    t_data = time step of data (td+1)
    t_steps = desired time step (ti+1 may or may not equal td+1)
        Default value of t_data = t_steps
    # t_data and t_steps must return a divisble integer number
    
    obsN = # of Observer [The number of observer is proportional the the 
                            data size]
    
    Returns 
    X = States 
    P = Covariance Matrix
    N_Occ = # of Occlusion Events that happen
    -------
    """
    # EKF propagation time
    t_idx = np.size(DT, 1)
    t_sidx = int(t_data/t_steps) # default value be 1
    """ ^ Above means we get results at each update iteration
        If desired in between tracking t_sidx > 1
    """
    
    # Moon Parameters
    DU = 389703 # Distance Units
    rb = 1737.1/DU
    rt = 0.01/DU
    pb = np.array([[1-mu],[0],[0]])
    
    # Measurement Uncertainity (Arc-Seconds to Radians)
    unc = np.array([26.7518, 82.63, 192.0118]) * (np.pi/(180*3600))
    
    # Storing Matrix 
    X = np.zeros([6, t_idx*t_sidx]) # 6 by 1 state
    P = np.zeros([6,6*t_idx*t_sidx]) # 6 by 6 covariance
    N_Occ = np.zeros([t_idx*t_sidx]) # # of Occlusion Storage
    
    for i in range(t_idx):
        Xi = Xi.reshape(-1,1) # Reshape Matrix every time to ensure column vector
        
        for j in range(obsN):
            # Sequential Update due to # of Observer
            do = DO[j*3:j*3+3, i].reshape(-1,1) # DO[i, j*3-2:j*3]
            dt = DT[0:3, i].reshape(-1,1) #DT[i, 0:3]
            check = OcclusionAngle(dt, do, rb, rt, pb)
            
            if check == True: # True = No-Occlusion
                Xi, Pi = AngleEKF(dt, do, Xi, Pi, unc[2]) # Update Measurements AngleEKF
                
            else: # False = Positve Occlusion
                N_Occ[i] += 1
            
            # Update Store Matrix
            X[:,[i]] = Xi[:]
            P[:,6*i:6*i+6] = Pi
        
        IC_prop = np.concatenate((Xi, Pi.reshape([36,1])))
        IC = IC_prop.reshape([42,]) # Reshape Matrix
        
        for k in range(t_sidx):
            sol = solve_ivp(First_Order_CR3BP, [0, t_steps], IC, args = (mu,), 
                            rtol = 1E-13, atol = 1e-13) # first_step = 1E-3, 
            
            # If t_sidx > 1 redefine inital condition
            IC = sol.y[0:,-1]
            
            # Update Xi and Pi for next time iteration
            Xi = sol.y[0:6,-1]
            Pi = np.reshape(sol.y[6:,-1], [6,6])
            
            # Store Values
            X[:,i] = Xi
            P[:,6*i:6*i+6] = Pi
        
    return X, P, N_Occ

In [68]:
### Angle Measurement Model
def AngleEKF(DT, DO, Xi, Pi, unc):
    """
    Parameters
    ----------
    DT = Target Data 
    DO = Observer Data
    Xi = Inital Conditon [6 by 1]
    Pi = Inital Covariance Matrix [6 by 6]
    unc = Measurement Uncertainity 
        Angles unit = arc 
        
    The H (measurement matrix) is calculated as the partial deriative of the
    measurements with respect to the states
        
    Returns: 
        Updated State and Covariance Matrix
        X and P based on Target Data
    -------
    """
    targX, obsX = DT, DO # Data Identification
    
    # Angle Calculation
    rho_obs_L1 = np.array([[0.83691513], [0], [0]]) - obsX # Same for gut measurement
    rho_obs_targ = targX - obsX
    
    # Angle Identification
    Lx,Ly,Lz = rho_obs_L1[0:]
    rhox,rhoy,rhoz = rho_obs_targ[0:]
    
    # Right Ascension and Declination for Measurement [2 by 1]
    alpha = np.arccos((Lx*rhox + Ly*rhoy)/(np.linalg.norm(rho_obs_L1)*np.sqrt(rhox**2 + rhoy**2)))
    decl = np.arccos((Lx*rhox + Lz*rhoz)/(np.sqrt(Lx**2 + Lz**2)*np.sqrt(rhox**2 + rhoz**2)))
    Meas = np.array([[alpha[0]],[decl[0]]])
    
    # Gut Check (z) using priori State
    tX = Xi[0:3]
    rho_gut_targ = tX - obsX
    
    rhox_gut, rhoy_gut, rhoz_gut = rho_gut_targ[0:]
    alpha_gut = np.arccos((Lx*rhox_gut + Ly*rhoy_gut)/(np.linalg.norm(rho_obs_L1)*np.sqrt(rhox_gut**2 + rhoy_gut**2)))
    decl_gut = np.arccos((Lx*rhox_gut + Lz*rhoz_gut)/(np.sqrt(Lx**2 + Lz**2)*np.sqrt(rhox_gut**2 + rhoz_gut**2)))
    z = np.array([[alpha_gut[0]],[decl_gut[0]]])
    
    # H Matrix Creation 
    # Parital Derative of alpha and decl with respect to rhox, rhoy, rhoz
    dalpha_drhox = -(rhoy*(Lx*rhoy - Ly*rhox))/((rhox**2 + rhoy**2)**(3/2)*(1 - (Lx*rhox + Ly*rhoy)**2/((rhox**2 + rhoy**2)*(Lx**2 + Ly**2 + Lz**2)))**(1/2)*(Lx**2 + Ly**2 + Lz**2)**(1/2))
    dalpha_drhoy = -(Ly*rhox**2 - Lx*rhoy*rhox)/((rhox**2 + rhoy**2)**(3/2)*(1 - (Lx*rhox + Ly*rhoy)**2/((rhox**2 + rhoy**2)*(Lx**2 + Ly**2 + Lz**2)))**(1/2)*(Lx**2 + Ly**2 + Lz**2)**(1/2))
    
    ddecl_drhox = -(rhoz*(Lx*rhoz - Lz*rhox))/((rhox**2 + rhoz**2)**(3/2)*(Lx**2 + Lz**2)**(1/2)*((Lx*rhoz - Lz*rhox)**2/((rhox**2 + rhoz**2)*(Lx**2 + Lz**2)))**(1/2)) 
    ddecl_drhoz = -(Lz*rhox**2 - Lx*rhoz*rhox)/((rhox**2 + rhoz**2)**(3/2)*(Lx**2 + Lz**2)**(1/2)*((Lx*rhoz - Lz*rhox)**2/((rhox**2 + rhoz**2)*(Lx**2 + Lz**2)))**(1/2))
    
    dangle = np.array([[dalpha_drhox[0], dalpha_drhoy[0], 0], [ddecl_drhox[0], 0, ddecl_drhoz[0]]]) # [2 by 3]
    drho_dX = np.array([[1,0,0,0,0,0], [0,1,0,0,0,0], [0,0,1,0,0,0]]) #[3 by 6]
    
    H_tilde = dangle @ drho_dX
    
    # Compute Kalman Gain
    R = unc**2
    K = Pi @ np.transpose(H_tilde) @ np.linalg.inv((H_tilde @ Pi @ np.transpose(H_tilde) + R)) #[6 by 2]
    
    # Compute Updated State and Covariance Matrix
    Xnew = Xi + K @ (Meas - z)
    Pnew = (np.eye(6) - K @ H_tilde) @ Pi @ np.transpose(np.eye(6) - K @ H_tilde) + (K*R) @ np.transpose(K)
    
    return Xnew, Pnew

In [69]:
### Lineraziation for First Order Propagation
def First_Order_CR3BP(t, X, mu):
    """
    EKF requires linearization of the state equation using first-order
    approximation (Taylor-Series)
    
    Combine Propagation of the state and covariance matrix
    Requires the creation of the F matrix
    
    Parameters
    ----------
    t : time
        solving ode of equation of motion.
    X : State of spacecraft + Covariance Matrix
        X = [x; y; z; vx; vy; vz] First Six
        P = [var(xx) cov(x,y) cov(x,z), cov(x,vx), cov(x,vy), cov(x,vz), var(yy), ...] [36 by 1]
    mu : gravtational parameter of system
        mu = constant of 1.215058560962404E-2

    Returns Xdot: [42, 1] Matrix of States and Covariance
    -------
    """
    x,y,z,vx,vy,vz = X[0:6] # Defining states
    
    # Length Parameter
    rr1 = x + mu
    rr2 = x - 1 + mu
    r1 = np.linalg.norm(np.array([rr1, y, z]))
    r2 = np.linalg.norm(np.array([rr2, y, z]))
    
    ### States Equation of Motion
    ax = 2*vy + x - ((1-mu)/r1**3) * (x+mu) - (mu/r2**3) * (x-1+mu)
    ay = y - 2*vx - ((1-mu)/r1**3) * y - (mu/r2**3) * y
    az = -((1-mu)/r1**3) * z - ((mu/r2**3)) * z 
    
    Xdot = np.array([vx,vy,vz,ax,ay,az])
    
    ### Covariance Propagation
    Pi = X[6:].reshape([6,6])
    
    # Building F_tilde Matrix
    O = np.zeros([3,3])
    I = np.identity(3)
    
    # A Matrix
    dx_dx = (mu-1)/r1**3 - mu/r2**3 + 3*mu*(2*rr2)*(rr2)/(2*r2**5) - 3*(2*rr1)*rr1*(mu-1)/(2*r1**5) + 1
    dx_dy = 3*y*mu*(rr2)/r2**5 - 3*y*rr1*(mu-1)/r1**5
    dx_dz = 3*z*mu*(rr2)/r2**5 - 3*z*rr1*(mu-1)/r1**5
    dy_dx = 3*y*mu*(2*rr2)/(2*r2**5) - 3*y*(2*rr1)*(mu-1)/(2*r1**5)
    dy_dy = (mu-1)/r1**3 - mu/r2**3 - 3*y**2*(mu-1)/r1**5 + 3*y**2*mu/r2**5 + 1
    dy_dz = 3*y*z*mu/r2**5 - 3*y*z*(mu-1)/r1**5
    dz_dx = 3*z*mu*(2*rr2)/(2*r2**5) - 3*z*(2*rr1)*(mu-1)/(2*r1**5)
    dz_dy = 3*y*z*mu/r2**5 - 3*y*z*(mu-1)/r1**5
    dz_dz = (mu-1)/r1**3 - mu/r2**3 - 3*z**2*(mu-1)/r1**5 + 3*z**2*mu/r2**5
    
    A = np.array([[dx_dx, dx_dy, dx_dz], [dy_dx, dy_dy, dy_dz], [dz_dx, dz_dy, dz_dz]])
    
    B = np.array([[0,2,0],[-2,0,0],[0,0,0]])
    
    Up = np.concatenate((O,I),1)
    Low = np.concatenate((A,B),1)
    
    F_tilde = np.concatenate((Up,Low),0)
    
    G = np.concatenate((O,I), 0)
    Q = 1E-12 * I
    
    # Covariance Propagation
    Pdot = F_tilde @ Pi + Pi @ np.transpose(F_tilde) + G @ Q @ np.transpose(G)
    Pdot = Pdot.reshape([36,])
    
    # Reshape Matirx for IVP
    Ydot = np.concatenate((Xdot,Pdot))
    
    return Ydot

In [70]:
# Occlusion Model when Moon blocks out the observers        
def OcclusionAngle(DT, DO, rb, rt, pb):
    """
    Parameters
    ----------
    DT = Target Data 
    DO = Observer Data
    rb = Radius of the main body (Default Moon)
    rt = Radius of Target Body
    pb = position of the body (Default: [1-mu, 0, 0])
    
    # rb, rt, pb are dependent on the system (CR3BP) and the units are
    dimensionalized

    Returns = Pass Criteria (True = 1) or Fail Criteria (False = 0)
    -------
    """
    
    # Length Calculation
    ts, os = DT, DO
    r_om = pb - os
    r_ot = ts - os
    
    # Angle Calculation
    beta = np.arcsin(rb/np.linalg.norm(r_om))
    gamma = np.arcsin(rt/np.linalg.norm(r_ot))
    alpha = np.arccos(np.dot(r_om.transpose(),r_ot)/(np.linalg.norm(r_om)*np.linalg.norm(r_ot)))
    
    # Convert to Angles if you want better repersentation
    # beta, gamma, alpha = np.array([beta, gamma, alpha]) * 180/np.pi
    
    if alpha > (beta+gamma):
        check = True
    else:
        check = False
        
    return check